# Imports

In [ ]:
import importlib
import utils.biopython_tools as bptools
import json
import pandas as pd

In [28]:
importlib.reload(bptools)

<module 'utils.biopython_tools' from '/home/markus/projects/riff_diff/ensemble_evaluator/ensemble_evaluator/utils/biopython_tools.py'>

In [9]:
motif_d = {"A": [1, 2, 3, 4, 5], "B": [6, 7, 8, 9]}
def convert_motifdict_to_list(motif_dict: dict) -> list: 
    return [(chain, res) for chain in motif_dict for res in motif_dict[chain]]

print(convert_motifdict_to_list(motif_d))

[('A', 1), ('A', 2), ('A', 3), ('A', 4), ('A', 5), ('B', 6), ('B', 7), ('B', 8), ('B', 9)]


# Test replacement

In [18]:
# get motif dicts:
motif_df = pd.read_json("/home/markus/projects/rfdiffusion_testing/motif_replacement/motif_res.json")
pose_motif = motif_df.loc["D33-C34-B26-A44_0001_0001"]["motif_residues"]

In [12]:
pose_p = "/home/markus/projects/rfdiffusion_testing/motif_replacement/D33-C34-B26-A44_0001_0001.pdb"
motif_pose_p = "/home/markus/projects/rfdiffusion_testing/motif_replacement/ref_fragment/D33-C34-B26-A44_0001_0001.pdb"

In [46]:
importlib.reload(bptools)
bptools.replace_motif_and_add_ligand(pose_p, motif_pose_p, pose_motif, pose_motif, new_pose="/home/markus/projects/rfdiffusion_testing/motif_replacement/D33-C34-B26-A44_0001_0001_replaced.pdb", ligand_chain="F")
#
#


4
5
6
7
8
9
10
21
22
23
24
25
26
27
38
39
40
41
42
43
44
53
54
55
56
57
58
59


'/home/markus/projects/rfdiffusion_testing/motif_replacement/D33-C34-B26-A44_0001_0001_replaced.pdb'

# Function for Contig reordering

In [15]:
pose_opts = "/home/mabr3112/anaconda3/envs/SE3nv/bin/python3 /home/mabr3112/RFdiffusion/run_inference.py diffuser.T=20 potentials.guide_scale=5 inference.num_designs=15 potentials.guiding_potentials=[\'type:monomer_ROG,weight:1000,min_dist:0.1\',\'type:substrate_contacts,weight:5\'] potentials.guide_decay='quadratic' 'contigmap.contigs=[3/A1-7/10/B1-7/8/C1-7/10/D1-7/37]' 'contigmap.inpaint_seq=[A1/A3/A4/A5/A6/A7/B1/B2/B3/B4/B5/B7/C1/C2/C3/C4/C6/C7/D1/D2/D4/D5/D6/D7]' potentials.substrate=EQU inference.output_prefix=/home/mabr3112/projects/riff_diff/ksi/diffusion/ksi_diff_small_v2_rog/rfdiffusion/output_pdbs//C2-A46-D50-B11 inference.input_pdb=/home/mabr3112/projects/riff_diff/ksi/paths/ksi_diff_small/pdb_in/C2-A46-D50-B11.pdb"
total_length = 200


In [24]:
full_contig_str = [x for x in pose_opts.split(" ") if x.startswith("'contigmap.contigs")][0]
contig_str = full_contig_str[full_contig_str.find("[")+1:full_contig_str.find("]")]
contigs = [x for x in contig_str.split("/") if x][1:-1]
new_contigs = "/".join([x if x[0].isalpha() else f"{x}-100" for x in contigs])
new_contig_str = full_contig_str.replace("/".join(contigs), new_contigs)
print(new_contigs)
print("/".join(contigs))

print(pose_opts.replace(full_contig_str, f"{new_contig_str} contigmap.length={total_length} "))

A1-7/10-100/B1-7/8-100/C1-7/10-100/D1-7
A1-7/10/B1-7/8/C1-7/10/D1-7
/home/mabr3112/anaconda3/envs/SE3nv/bin/python3 /home/mabr3112/RFdiffusion/run_inference.py diffuser.T=20 potentials.guide_scale=5 inference.num_designs=15 potentials.guiding_potentials=['type:monomer_ROG,weight:1000,min_dist:0.1','type:substrate_contacts,weight:5'] potentials.guide_decay='quadratic' 'contigmap.contigs=[3/A1-7/10-100/B1-7/8-100/C1-7/10-100/D1-7/37]' contigmap.length=200  'contigmap.inpaint_seq=[A1/A3/A4/A5/A6/A7/B1/B2/B3/B4/B5/B7/C1/C2/C3/C4/C6/C7/D1/D2/D4/D5/D6/D7]' potentials.substrate=EQU inference.output_prefix=/home/mabr3112/projects/riff_diff/ksi/diffusion/ksi_diff_small_v2_rog/rfdiffusion/output_pdbs//C2-A46-D50-B11 inference.input_pdb=/home/mabr3112/projects/riff_diff/ksi/paths/ksi_diff_small/pdb_in/C2-A46-D50-B11.pdb


In [ ]:
def overwrite_linker_length(pose_opts: str, total_length:int, max_linker_length:int=100) -> str:
    '''overwrites linker length and allows linkers to be of any length (with at least the provided linker length)'''
    full_contig_str = [x for x in pose_opts.split(" ") if x.startswith("'contigmap.contigs")][0]
    contig_str = full_contig_str[full_contig_str.find("[")+1:full_contig_str.find("]")]
    contigs = [x for x in contig_str.split("/") if x][1:-1]
    new_contigs = "/".join([x if x[0].isalpha() else f"{x}-{max_linker_length}" for x in contigs])
    new_contig_str = full_contig_str.replace("/".join(contigs), new_contigs)
    return pose_opts.replace(full_contig_str, f"{new_contig_str} contigmap.length={total_length} ")